<a href="https://colab.research.google.com/github/svedison/JDRE-Research/blob/main/CodetoObtainMedicalNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

In [ ]:
#if you only use the NOTEEVENTS.csv file
import pandas as pd

# Load the NOTEEVENTS.csv file (adjust path as necessary)
df_notes = pd.read_csv("NOTEEVENTS 2.csv")

# Display the first few rows to understand structure (optional)
#print(df_notes.head())

# Select columns of interest
columns_to_keep = ['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CATEGORY', 'DESCRIPTION', 'TEXT']
df_notes = df_notes[columns_to_keep]

# Optional: Filter by specific note types (e.g., discharge summaries only)
df_notes = df_notes[df_notes['CATEGORY'] == 'Discharge summary']

# Optional: Drop rows with missing TEXT or HADM_ID (depends on your needs)
df_notes = df_notes.dropna(subset=['TEXT', 'HADM_ID'])

# Optional: Combine multiple notes per HADM_ID or SUBJECT_ID if needed
# Example: Combine all notes per HADM_ID into one string
combined_notes = df_notes.groupby(['SUBJECT_ID', 'HADM_ID'])['TEXT'].apply(lambda x: '\n\n'.join(x)).reset_index()

# Save to CSV or use in-memory
combined_notes.to_csv('combined_medical_notes.csv', index=False)

print("Combined notes saved to 'combined_medical_notes.csv'")

notes_df = pd.read_csv('combined_medical_notes.csv')
print(notes_df.head())

In [ ]:
import pandas as pd

In [ ]:
#convert from .csv.gz to csv
df = pd.read_csv('PATIENTS.csv.gz')
df.to_csv('PATIENTS.csv', index=False)

In [ ]:
#get base path
import os
full_path = '/Users/emaseibert/physionet.org/files/mimiciii/1.4/PATIENTS.csv'
base_path = os.path.dirname(full_path)
print("Base path is:", base_path)

In [ ]:
# Load relevant tables
path_base = base_path
patients = pd.read_csv(path_base + '/PATIENTS.csv')
admissions = pd.read_csv(path_base + '/ADMISSIONS.csv')
noteevents = pd.read_csv(path_base + '/NOTEEVENTS 2.csv', low_memory=False)

In [ ]:
#see the different columns
print(patients.columns)
print(admissions.columns)
print(noteevents.columns)

In [ ]:
#observe that patients columns are all lower-cased so capitalize before merging
# Convert all column names to uppercase
patients.columns = patients.columns.str.upper()

print("\nDataFrame columns after conversion:")
print(patients.columns)

In [ ]:
# Drop deidentified text (if needed)
noteevents = noteevents.dropna(subset=['TEXT'])

# Join NOTEEVENTS with ADMISSIONS on SUBJECT_ID and HADM_ID
notes_admissions = pd.merge(noteevents, admissions, on=['SUBJECT_ID', 'HADM_ID'], how='left')

# Join with PATIENTS on SUBJECT_ID
full_notes = pd.merge(notes_admissions, patients, on='SUBJECT_ID', how='left')

# Keep only desired columns
final_columns = [
    'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CATEGORY', 'DESCRIPTION', 'TEXT',
    'ADMITTIME', 'DISCHTIME', 'DIAGNOSIS',
    'GENDER', 'DOD'
]
final_notes = full_notes[final_columns]

# Remove notes without admission info
final_notes = final_notes.dropna(subset=['HADM_ID'])

# Save to file
final_notes.to_csv('combined_mimic_medical_notes.csv', index=False)

NOTEEVENTS.csv contains the clinical notes.
ROW_ID (unique row identifier)
SUBJECT_ID (patient identifier)
HADM_ID (unique hospital admission id)
CHARTDATE (date note written)
CHARTTIME (time note written)
STORETIME (not used)
CATEGORY (type note)
DESCRIPTION (note description)
CGID (not used)
ISERROR (not used)
TEXT (full free-text clinical note)

Combine NOTEEVENTS.csv with PATIENTS.csv and ADMISSIONS.csv to add gender and dod from patients.csv; hospital mortality, admission type, admit time, discharge time from admissions.csv.

In [ ]:
#Make a dataframe from the new csv file with merged columns
notes_df = pd.read_csv('combined_mimic_medical_notes.csv')

notes_df = notes_df[notes_df['TEST'].notnull()]
clean_df = notes_df[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CATEGORY', 'DESCRIPTION', 'TEXT',
    'ADMITTIME', 'DISCHTIME', 'DIAGNOSIS',
    'GENDER', 'DOD']]

print(clean_df.head())

#SUBJECT_ID: unique identifier for each patient (all)
#HADM_ID: unique identifier for each hospital admission (admissions, noteevents)
#CHARTDATE: date when clinical note was added/recorded (noteevents)
#CATEGORY: category of clincial note - physician note, nursing note, discharge summary, etc (noteevents)
#DESCRIPTION: short label/description of clinical note added (noteevents)
#TEXT: full free-text content of clinical note (noteevents)
#ADMITTIME: time stamp when patient was admitted (admissions)
#DISCHTIME: time stamp when patient was discharged from the hospital (admissions)
#DIAGNOSIS: general diagnosis given at discharge (admissions)
#GENDER: patient's gender male or female (patients)
#DOD: date patient died if applicable else null (patients)